In [2]:
from dask_jobqueue import SLURMCluster 
from dask.distributed import Client 
  
cluster = SLURMCluster(cores=28,name='make_profiles',walltime='00:30:00',job_extra=['--constraint=HSW24','--exclusive','--nodes=1'],memory='120GB',interface='ib0') 

cluster.scale(102)   

/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/bokeh/core.py:74: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 13] Permission denied
  warnings.warn("\n" + msg)


In [3]:
from dask.distributed import Client
client = Client(cluster)
client

Client Scheduler: tcp://172.30.100.2:40427 Dashboard: http://172.30.100.2:44887/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [4]:
!squeue -u albert7a

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           7562459      allc make_pro albert7a PD       0:00      1 (None)
           7562460      allc make_pro albert7a PD       0:00      1 (None)
           7562461      allc make_pro albert7a PD       0:00      1 (None)
           7562462      allc make_pro albert7a PD       0:00      1 (None)
           7562183       all nemo_occ albert7a PD       0:00     49 (Resources)


In [5]:
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import numpy.ma as ma
import numpy as np
import netCDF4 as nc4
import xarray as xr
import sys
sys.path.insert(0,"/home/albert7a/lib/python/pypago")
import os
import matplotlib
import pylab as plt
import numpy as np
import pyproj

%matplotlib inline
%autosave 60

/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/xarray/core/merge.py:16: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


Autosaving every 60 seconds


In [6]:
#data and grid
gridname='/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-I/eNATL60_coordinates_v3.nc4'
dgrid=xr.open_dataset(gridname)
navlon=dgrid['nav_lon']
navlat=dgrid['nav_lat']

wname='/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-BLBT02-S/00529201-00540000/eNATL60-BLBT02_1h_20090903_20090907_gridW_20090907-20090907.nc'
dw=xr.open_dataset(wname,chunks={'x':1000,'y':1000,'time_counter':1,'depthw':1})
w=dw['vovecrtz']
depthw=dw['depthw']

In [7]:
ee = pyproj.Geod(ellps='GRS80')

In [8]:
def section_from_debut_fin_points(lon_ar7w_debut,lat_ar7w_debut,lon_ar7w_fin,lat_ar7w_fin,navlon,navlat,var):
    
    ee = pyproj.Geod(ellps='GRS80') # ellipsoid used for distance calculation
    
    def distance(lat1,lon1,lat2,lon2,geoid):
        az12,az21,dist = geoid.inv(lon1,lat1,lon2,lat2)
        return dist

    def loclatlon(navlon,navlat,lon,lat):
        ind_debut=np.where((navlon>lon-0.1)&(navlon<lon+0.1)&(navlat>lat-0.1)&(navlat<lat+0.1))
        dis=np.empty(len(ind_debut[0]))
        for ii in range(0,len(ind_debut[0])):
            dis[ii]=np.abs(distance(lat,lon,navlat[ind_debut[0][ii],ind_debut[1][ii]],navlon[ind_debut[0][ii],ind_debut[1][ii]],ee))
        ind=np.argmin(dis)
        i_debut=ind_debut[0][ind]
        j_debut=ind_debut[1][ind]
    
        return i_debut,j_debut
 
    def secfromdebuttofin(i_debut,j_debut,i_fin,j_fin):
        off=1
        fromi=np.atleast_1d(i_debut)+off
        fromj=np.atleast_1d(j_debut)+off
        toi=np.atleast_1d(i_fin)+off
        toj=np.atleast_1d(j_fin)+off

        veci=fromi
        vecj=fromj

        a=(toj-fromj)/((toi-fromi).astype(np.float))
        b=toj-toi*a

        while not((veci[-1]==toi) & (vecj[-1]==toj)):
            newi=veci[-1]+np.sign(toi-fromi)
            newj=vecj[-1]+np.sign(toj-fromj)
            y=a*newi+b
            x=(newj-b)/(a.astype(np.float))

            if ((np.abs(x-veci[-1]))>=(np.abs(y-vecj[-1]))):
                veci=np.append(veci,newi)
                vecj=np.append(vecj,vecj[-1])
            else:
                veci=np.append(veci,veci[-1])
                vecj=np.append(vecj,newj)

        veci=veci.astype(np.int)-off
        vecj=vecj.astype(np.int)-off

        dis=np.empty(len(veci))
        for ii in range(0,len(veci)):
            dis[ii]=np.abs(distance(navlat[veci[0],vecj[0]],navlon[veci[0],vecj[0]],navlat[veci[ii],vecj[ii]],navlon[veci[ii],vecj[ii]],ee))
    
        return veci,vecj,dis
            
    i_debut,j_debut=loclatlon(navlon,navlat,lon_ar7w_debut,lat_ar7w_debut)
    i_fin,j_fin=loclatlon(navlon,navlat,lon_ar7w_fin,lat_ar7w_fin)

    veci,vecj,dis=secfromdebuttofin(i_debut,j_debut,i_fin,j_fin)

    if np.ndim(var) == 4:
        [nt,nk,nj,ni]=var.shape
        var_sec=np.empty([nt,nk,len(veci)])
        for ii in range(0,len(veci)):
            var_sec[:,:,ii]=var[:,:,veci[ii],vecj[ii]]
    if np.ndim(var) == 3:
        [nt,nj,ni]=var.shape
        var_sec=np.empty([nt,len(veci)])
        for ii in range(0,len(veci)):
            var_sec[:,ii]=var[:,veci[ii],vecj[ii]]
    
    return veci,vecj,dis,var_sec

In [ ]:
#Labrador section beginning and end point

lon_ar7w_debut=np.array(-56).astype(np.float)
lon_ar7w_fin=np.array(-48).astype(np.float)

lat_ar7w_debut=np.array(53.5).astype(np.float)
lat_ar7w_fin=np.array(60.5).astype(np.float)

veci,vecj,dis,wlabrador=section_from_debut_fin_points(lon_ar7w_debut,lat_ar7w_debut,lon_ar7w_fin,lat_ar7w_fin,navlon,navlat,w.mean(dim='time_counter'))
wlabrador_smooth=np.ones(wlabrador.shape)
wlabrador_vert_variance=np.ones(wlabrador.shape)

delta=1e-6
for ii in range(len(veci)):
    profil=wlabrador[:,ii]
    window_len=3
    s=np.r_[profil[window_len-1:0:-1],profil,profil[-2:-window_len-1:-1]]
    whanning=np.hanning(window_len)
    yhanning=np.convolve(whanning/whanning.sum(),s,mode='valid')
    wlabrador_smooth[:,ii]=yhanning[0:300]
    wlabrador_vert_variance[:,ii]=np.sqrt(((wlabrador[:,ii]-wlabrador_smooth[:,ii])**2)/((wlabrador[:,ii]+delta)**2))


ma_wlabrador = np.ma.array (wlabrador, mask=np.isnan(wlabrador))
ma_wlabrador_smooth = np.ma.array (wlabrador_smooth, mask=np.isnan(wlabrador_smooth))
ma_wlabrador_vert_variance = np.ma.array (wlabrador_vert_variance, mask=np.isnan(wlabrador_vert_variance))
matplotlib.rcParams.update({'font.size': 22})

plt.figure(figsize=(30,40))

ax=plt.subplot(3,1,1)
cmap2 = plt.cm.seismic
cmap2.set_under('b')
cmap2.set_over('r')
pcolor=plt.pcolor(1e-3*dis,depthw,86400*ma_wlabrador[:,:],cmap=cmap2,vmin=-10,vmax=10)
plt.gca().invert_yaxis()
cbar = plt.colorbar(pcolor,orientation='horizontal',shrink=0.75,label='m/day')
ax.set_ylim(3800,0)
#ax.set_xlim(100,900)
plt.xlabel('distance to Canadian land in km')
plt.ylabel('depth in m')
plt.title('Vertical Velocity Labrador cross-section')

ax=plt.subplot(3,1,2)
cmap2 = plt.cm.seismic
cmap2.set_under('b')
cmap2.set_over('r')
pcolor=plt.pcolor(1e-3*dis,depthw,86400*(ma_wlabrador[:,:]-ma_wlabrador_smooth[:,:]),cmap=cmap2,vmin=-10,vmax=10)
plt.gca().invert_yaxis()
cbar = plt.colorbar(pcolor,orientation='horizontal',shrink=0.75,label='m/day')
ax.set_ylim(3800,0)
#ax.set_xlim(100,900)
plt.xlabel('distance to Canadian land in km')
plt.ylabel('depth in m')
plt.title('De-smoothed Vertical Velocity Labrador cross-section')

ax=plt.subplot(3,1,3)
cmap2 = plt.cm.seismic
cmap2.set_under('b')
cmap2.set_over('r')
pcolor=plt.pcolor(1e-3*dis,depthw,86400*np.abs(ma_wlabrador[:,:]-ma_wlabrador_smooth[:,:]),cmap='YlOrBr',vmin=0,vmax=10)
plt.gca().invert_yaxis()
cbar = plt.colorbar(pcolor,orientation='horizontal',shrink=0.75,label='rms')
ax.set_ylim(3800,0)
#ax.set_xlim(100,900)
plt.xlabel('distance to Canadian land in km')
plt.ylabel('depth in m')
plt.title('Vertical variance Labrador cross-section')

distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)


In [ ]:
#icescotland section beginning and end point

lon_icescotland_debut=np.array(-14.205).astype(np.float)
lon_icescotland_fin=np.array(-3.319).astype(np.float)

lat_icescotland_debut=np.array(64.72).astype(np.float)
lat_icescotland_fin=np.array(58.988).astype(np.float)

veci,vecj,dis,wicescotland=section_from_debut_fin_points(lon_icescotland_debut,lat_icescotland_debut,lon_icescotland_fin,lat_icescotland_fin,navlon,navlat,w)
wicescotland_smooth=np.ones(wicescotland.shape)

for ii in range(len(veci)):
    profil=wicescotland[0,:,ii]
    window_len=3
    s=np.r_[profil[window_len-1:0:-1],profil,profil[-2:-window_len-1:-1]]
    whanning=np.hanning(window_len)
    yhanning=np.convolve(whanning/whanning.sum(),s,mode='valid')
    wicescotland_smooth[0,:,ii]=yhanning[0:75]

ma_wicescotland = np.ma.array (wicescotland, mask=np.isnan(wicescotland))
ma_wicescotland_smooth = np.ma.array (wicescotland_smooth, mask=np.isnan(wicescotland_smooth))

matplotlib.rcParams.update({'font.size': 22})

plt.figure(figsize=(30,25))

ax=plt.subplot(2,1,1)
cmap2 = plt.cm.seismic
cmap2.set_under('b')
cmap2.set_over('r')
pcolor=plt.pcolor(1e-3*dis,depthw,86400*ma_wicescotland[0,:,:],cmap=cmap2,vmin=-10,vmax=10)
plt.gca().invert_yaxis()
cbar = plt.colorbar(pcolor,orientation='horizontal',shrink=0.75,label='m/day')
ax.set_ylim(1500,0)
#ax.set_xlim(100,900)
plt.xlabel('distance to Iceland in km')
plt.ylabel('depth in m')
plt.title('Vertical Velocity cross-section');

ax=plt.subplot(2,1,2)
cmap2 = plt.cm.seismic
cmap2.set_under('b')
cmap2.set_over('r')
pcolor=plt.pcolor(1e-3*dis,depthw,86400*np.abs(ma_wicescotland[0,:,:]-ma_wicescotland_smooth[0,:,:]),cmap='YlOrBr',vmin=0,vmax=10)
plt.gca().invert_yaxis()
cbar = plt.colorbar(pcolor,orientation='horizontal',shrink=0.75,label='rms')
ax.set_ylim(1500,0)
#ax.set_xlim(100,900)
plt.xlabel('distance to Iceland in km')
plt.ylabel('depth in m')
plt.title('Indicator for grid noise in Vertical velocities cross-section');